# E-commerce Business Analytics Report

A comprehensive analysis of e-commerce sales data focusing on business performance metrics, customer satisfaction, and operational efficiency.

## Table of Contents

1. [Introduction and Business Objectives](#introduction)
2. [Data Loading and Configuration](#data-loading)
3. [Data Dictionary](#data-dictionary)
4. [Data Preparation and Transformation](#data-preparation)
5. [Business Metrics Analysis](#business-metrics)
   - [Revenue Performance Analysis](#revenue-analysis)
   - [Product Category Performance](#product-analysis)
   - [Geographic Performance Analysis](#geographic-analysis)
   - [Customer Experience Analysis](#customer-analysis)
6. [Summary of Key Observations](#summary)

<a id='introduction'></a>
## 1. Introduction and Business Objectives

This analysis examines e-commerce sales data to provide actionable insights for business decision-making. The analysis is designed to be configurable, allowing for flexible time period comparisons and reusable across different datasets.

### Business Objectives

- **Revenue Performance**: Analyze total revenue, growth trends, and order patterns
- **Product Strategy**: Identify top-performing categories and revenue drivers
- **Geographic Insights**: Understand regional performance variations
- **Customer Satisfaction**: Evaluate delivery performance and review metrics
- **Operational Efficiency**: Assess delivery times and fulfillment quality

### Analysis Configuration

The analysis parameters can be adjusted below to analyze different time periods:

In [ ]:
# Analysis Configuration Parameters
# Modify these parameters to analyze different time periods

ANALYSIS_YEAR = 2023          # Primary year to analyze
COMPARISON_YEAR = 2022        # Comparison year for growth calculations (or None)
ANALYSIS_MONTH = None         # Specific month to analyze (1-12), or None for full year
DATA_PATH = 'ecommerce_data/' # Path to data directory

print(f"Analysis Period: {ANALYSIS_YEAR}")
print(f"Comparison Period: {COMPARISON_YEAR if COMPARISON_YEAR else 'None'}")
print(f"Month Filter: {ANALYSIS_MONTH if ANALYSIS_MONTH else 'Full Year'}")

<a id='data-loading'></a>
## 2. Data Loading and Configuration

Loading required libraries and initializing the analysis framework with custom modules for data loading and metrics calculation.

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

# Import custom modules
from data_loader import EcommerceDataLoader, load_and_process_data
from business_metrics import (
    BusinessMetricsCalculator,
    MetricsVisualizer,
    print_metrics_summary
)

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Configure matplotlib for inline display in Jupyter
try:
    get_ipython().run_line_magic('matplotlib', 'inline')
except NameError:
    # If not in Jupyter environment, use standard backend
    plt.ion()

print("Libraries loaded successfully")

In [ ]:
# Load and process all datasets
loader, processed_data = load_and_process_data(DATA_PATH)

# Display data summary
data_summary = loader.get_data_summary()
print("\nDataset Summary:")
print("=" * 70)
for dataset, info in data_summary.items():
    print(f"\n{dataset.upper()}:")
    print(f"  Rows: {info['rows']:,}")
    print(f"  Columns: {info['columns']}")
    print(f"  Memory Usage: {info['memory_usage_mb']:.1f} MB")
    if info['date_range']:
        print(f"  Date Range: {info['date_range']['start'].date()} to {info['date_range']['end'].date()}")

<a id='data-dictionary'></a>
## 3. Data Dictionary

### Core Data Tables

**Orders Dataset**
- `order_id`: Unique identifier for each order
- `customer_id`: Unique identifier for the customer
- `order_status`: Current status (delivered, shipped, canceled, etc.)
- `order_purchase_timestamp`: Date and time when order was placed
- `order_delivered_customer_date`: Date and time when order was delivered

**Order Items Dataset**
- `order_id`: Links to orders table
- `product_id`: Unique identifier for the product
- `price`: Item price in USD (excluding shipping)
- `freight_value`: Shipping cost in USD

**Products Dataset**
- `product_id`: Unique identifier for each product
- `product_category_name`: Product category classification

**Customers Dataset**
- `customer_id`: Unique identifier for each customer
- `customer_state`: State abbreviation (e.g., CA, TX, NY)
- `customer_city`: City name

**Reviews Dataset**
- `order_id`: Links to orders table
- `review_score`: Customer rating on 1-5 scale (5 = best)

### Key Business Metrics Definitions

**Revenue Metrics**
- **Total Revenue**: Sum of all item prices for delivered orders
- **Average Order Value (AOV)**: Total revenue divided by number of unique orders
- **Revenue Growth Rate**: Year-over-year percentage change in revenue

**Customer Metrics**
- **Review Score**: Average customer satisfaction rating (1-5 scale)
- **Delivery Days**: Time between order placement and delivery to customer
- **Fast Delivery**: Orders delivered within 3 days or less

**Product Metrics**
- **Revenue Share**: Percentage of total revenue contributed by a category
- **Top Categories**: Categories ranked by total revenue

<a id='data-preparation'></a>
## 4. Data Preparation and Transformation

Creating comprehensive sales datasets for the analysis and comparison periods. The data is filtered based on the configuration parameters and includes all relevant information from orders, products, customers, and reviews.

In [ ]:
# Create sales dataset for analysis period
sales_data_current = loader.create_sales_dataset(
    year_filter=ANALYSIS_YEAR,
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'
)

print(f"Analysis Dataset ({ANALYSIS_YEAR}):")
print(f"  Total Records: {len(sales_data_current):,}")
print(f"  Unique Orders: {sales_data_current['order_id'].nunique():,}")
print(f"  Total Revenue: ${sales_data_current['price'].sum():,.2f}")
if len(sales_data_current) > 0:
    print(f"  Date Range: {sales_data_current['order_purchase_timestamp'].min().date()} to {sales_data_current['order_purchase_timestamp'].max().date()}")

# Display sample of the dataset
print(f"\nSample Data (first 5 records):")
display(sales_data_current[[
    'order_id', 'price', 'purchase_year', 'purchase_month',
    'product_category_name', 'customer_state', 'review_score', 'delivery_days'
]].head())

In [ ]:
# Create comparison dataset if comparison year is specified
if COMPARISON_YEAR:
    sales_data_comparison = loader.create_sales_dataset(
        year_filter=COMPARISON_YEAR,
        month_filter=ANALYSIS_MONTH,
        status_filter='delivered'
    )
    
    print(f"Comparison Dataset ({COMPARISON_YEAR}):")
    print(f"  Total Records: {len(sales_data_comparison):,}")
    print(f"  Unique Orders: {sales_data_comparison['order_id'].nunique():,}")
    print(f"  Total Revenue: ${sales_data_comparison['price'].sum():,.2f}")
    
    # Create combined dataset for year-over-year analysis
    combined_data = pd.concat([sales_data_current, sales_data_comparison], ignore_index=True)
else:
    combined_data = sales_data_current
    print("No comparison year specified")

print(f"\nCombined Dataset: {len(combined_data):,} records")

<a id='business-metrics'></a>
## 5. Business Metrics Analysis

Comprehensive analysis of key business performance indicators using the BusinessMetricsCalculator class.

In [ ]:
# Initialize metrics calculator with combined dataset
metrics_calc = BusinessMetricsCalculator(combined_data)

# Generate comprehensive business report
business_report = metrics_calc.generate_comprehensive_report(
    current_year=ANALYSIS_YEAR,
    previous_year=COMPARISON_YEAR
)

# Print executive summary
print_metrics_summary(business_report)

<a id='revenue-analysis'></a>
### 5.1 Revenue Performance Analysis

Analyzing overall revenue trends, growth patterns, and key financial performance indicators.

In [ ]:
# Extract revenue metrics from report
revenue_metrics = business_report['revenue_metrics']

print(f"DETAILED REVENUE ANALYSIS - {ANALYSIS_YEAR}")
print("=" * 70)
print(f"\nFinancial Performance:")
print(f"  Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"  Total Orders: {revenue_metrics['total_orders']:,}")
print(f"  Total Items Sold: {revenue_metrics['total_items_sold']:,}")
print(f"  Average Order Value: ${revenue_metrics['average_order_value']:,.2f}")

if COMPARISON_YEAR and 'revenue_growth_rate' in revenue_metrics:
    print(f"\nYear-over-Year Comparison vs {COMPARISON_YEAR}:")
    print(f"  Revenue Growth: {revenue_metrics['revenue_growth_rate']:+.2f}%")
    print(f"  Order Growth: {revenue_metrics['order_growth_rate']:+.2f}%")
    print(f"  AOV Growth: {revenue_metrics['aov_growth_rate']:+.2f}%")
    
    print(f"\nPrevious Year ({COMPARISON_YEAR}):")
    print(f"  Total Revenue: ${revenue_metrics['previous_year_revenue']:,.2f}")
    print(f"  Total Orders: {revenue_metrics['previous_year_orders']:,}")
    print(f"  Average Order Value: ${revenue_metrics['previous_year_aov']:,.2f}")

In [ ]:
# Visualize monthly revenue trend
visualizer = MetricsVisualizer(business_report)
revenue_fig = visualizer.plot_revenue_trend(figsize=(14, 6))
plt.show()

# Display monthly trends table
monthly_trends = business_report['monthly_trends']
print(f"\nMonthly Revenue Breakdown for {ANALYSIS_YEAR}:")
display(monthly_trends[['month', 'revenue', 'orders', 'avg_order_value', 'revenue_growth']].round(2))

In [ ]:
# Monthly performance insights
monthly_trends = business_report['monthly_trends']

best_month = monthly_trends.loc[monthly_trends['revenue'].idxmax()]
worst_month = monthly_trends.loc[monthly_trends['revenue'].idxmin()]
avg_monthly_growth = monthly_trends['revenue_growth'].mean()

print(f"Monthly Performance Insights:")
print(f"  Best Revenue Month: Month {int(best_month['month'])} (${best_month['revenue']:,.0f})")
print(f"  Lowest Revenue Month: Month {int(worst_month['month'])} (${worst_month['revenue']:,.0f})")
print(f"  Average Monthly Growth: {avg_monthly_growth:.2f}%")
print(f"  Revenue Volatility (Std Dev): ${monthly_trends['revenue'].std():,.0f}")

<a id='product-analysis'></a>
### 5.2 Product Category Performance

Understanding which product categories drive the most revenue and identifying growth opportunities.

In [ ]:
# Product category analysis
product_data = business_report['product_performance']

if 'error' not in product_data:
    print(f"TOP PRODUCT CATEGORIES - {ANALYSIS_YEAR}")
    print("=" * 70)
    
    top_categories = product_data['top_categories']
    
    # Display top categories
    print(f"\n{'Category':<30} {'Revenue':>15} {'Share':>10} {'Orders':>10}")
    print("-" * 70)
    for idx, row in top_categories.iterrows():
        print(f"{row['product_category_name']:<30} ${row['total_revenue']:>14,.0f} {row['revenue_share']:>9.1f}% {row['unique_orders']:>9,}")
    
    # Category insights
    total_categories = len(product_data['all_categories'])
    top_5_share = top_categories.head(5)['revenue_share'].sum()
    
    print(f"\nProduct Category Insights:")
    print(f"  Total Product Categories: {total_categories}")
    print(f"  Top 5 Categories Revenue Share: {top_5_share:.1f}%")
    print(f"  Market Concentration: {'High' if top_5_share > 70 else 'Moderate' if top_5_share > 50 else 'Diversified'}")
else:
    print("Product category data not available for analysis")

In [ ]:
# Visualize category performance
if 'error' not in product_data:
    category_fig = visualizer.plot_category_performance(top_n=10, figsize=(14, 8))
    plt.show()

<a id='geographic-analysis'></a>
### 5.3 Geographic Performance Analysis

Analyzing sales performance across different geographic regions to identify market opportunities and regional trends.

In [ ]:
# Geographic analysis
geo_data = business_report['geographic_performance']

if 'error' not in geo_data.columns:
    print(f"GEOGRAPHIC PERFORMANCE - {ANALYSIS_YEAR}")
    print("=" * 70)
    
    # Display top 10 states
    top_states = geo_data.head(10)
    print(f"\n{'State':<8} {'Revenue':>15} {'Orders':>10} {'Avg Order Value':>18}")
    print("-" * 70)
    for idx, row in top_states.iterrows():
        print(f"{row['state']:<8} ${row['revenue']:>14,.0f} {row['orders']:>9,} ${row['avg_order_value']:>17,.0f}")
    
    # Geographic insights
    total_states = len(geo_data)
    top_5_revenue = top_states.head(5)['revenue'].sum()
    total_revenue = geo_data['revenue'].sum()
    top_5_share = (top_5_revenue / total_revenue) * 100
    
    highest_aov_state = geo_data.loc[geo_data['avg_order_value'].idxmax()]
    
    print(f"\nGeographic Insights:")
    print(f"  States with Sales: {total_states}")
    print(f"  Top 5 States Revenue Share: {top_5_share:.1f}%")
    print(f"  Highest AOV State: {highest_aov_state['state']} (${highest_aov_state['avg_order_value']:,.0f})")
else:
    print("Geographic data not available for analysis")

In [ ]:
# Visualize geographic heatmap
if 'error' not in geo_data.columns:
    geo_fig = visualizer.plot_geographic_heatmap()
    geo_fig.show()

<a id='customer-analysis'></a>
### 5.4 Customer Experience Analysis

Evaluating customer satisfaction through review scores and delivery performance metrics.

In [ ]:
# Customer satisfaction analysis
satisfaction_metrics = business_report['customer_satisfaction']

if 'error' not in satisfaction_metrics:
    print(f"CUSTOMER SATISFACTION ANALYSIS - {ANALYSIS_YEAR}")
    print("=" * 70)
    print(f"\nReview Score Metrics:")
    print(f"  Average Review Score: {satisfaction_metrics['avg_review_score']:.2f}/5.0")
    print(f"  Total Reviews: {satisfaction_metrics['total_reviews']:,}")
    print(f"  5-Star Reviews: {satisfaction_metrics['score_5_percentage']:.1f}%")
    print(f"  4+ Star Reviews: {satisfaction_metrics['score_4_plus_percentage']:.1f}%")
    print(f"  Low Satisfaction (1-2 stars): {satisfaction_metrics['score_1_2_percentage']:.1f}%")
    
    # Satisfaction level assessment
    avg_score = satisfaction_metrics['avg_review_score']
    if avg_score >= 4.5:
        satisfaction_level = 'Excellent'
    elif avg_score >= 4.0:
        satisfaction_level = 'Good'
    elif avg_score >= 3.5:
        satisfaction_level = 'Fair'
    else:
        satisfaction_level = 'Needs Improvement'
    
    print(f"\nOverall Satisfaction Level: {satisfaction_level}")
else:
    print("Customer satisfaction data not available for analysis")

In [ ]:
# Visualize review distribution
if 'error' not in satisfaction_metrics:
    review_fig = visualizer.plot_review_distribution(figsize=(12, 6))
    plt.show()

In [ ]:
# Delivery performance analysis
delivery_metrics = business_report['delivery_performance']

if 'error' not in delivery_metrics:
    print(f"DELIVERY PERFORMANCE ANALYSIS - {ANALYSIS_YEAR}")
    print("=" * 70)
    print(f"\nDelivery Time Metrics:")
    print(f"  Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"  Median Delivery Time: {delivery_metrics['median_delivery_days']:.1f} days")
    print(f"  Fast Delivery (3 days or less): {delivery_metrics['fast_delivery_percentage']:.1f}%")
    print(f"  Slow Delivery (over 7 days): {delivery_metrics['slow_delivery_percentage']:.1f}%")
    
    # Delivery performance assessment
    avg_delivery = delivery_metrics['avg_delivery_days']
    if avg_delivery <= 3:
        delivery_rating = 'Excellent'
    elif avg_delivery <= 5:
        delivery_rating = 'Good'
    elif avg_delivery <= 7:
        delivery_rating = 'Fair'
    else:
        delivery_rating = 'Needs Improvement'
    
    print(f"\nDelivery Performance Rating: {delivery_rating}")
else:
    print("Delivery performance data not available for analysis")

In [ ]:
# Analyze relationship between delivery speed and satisfaction
if 'error' not in satisfaction_metrics and 'error' not in delivery_metrics:
    print(f"\nDelivery-Satisfaction Relationship:")
    print(f"  Average Review Score: {satisfaction_metrics['avg_review_score']:.2f}/5.0")
    print(f"  Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"  Fast Delivery Rate: {delivery_metrics['fast_delivery_percentage']:.1f}%")
    
    # Analyze delivery speed categories with review scores
    from data_loader import categorize_delivery_speed
    
    delivery_satisfaction = sales_data_current.copy()
    delivery_satisfaction['delivery_category'] = delivery_satisfaction['delivery_days'].apply(categorize_delivery_speed)
    
    speed_review_analysis = delivery_satisfaction.groupby('delivery_category')['review_score'].agg(['mean', 'count']).round(2)
    print(f"\nReview Scores by Delivery Speed:")
    display(speed_review_analysis)

In [ ]:
# Order status distribution
status_dist = business_report['order_status_distribution']

if 'error' not in status_dist.columns:
    print(f"\nORDER STATUS DISTRIBUTION - {ANALYSIS_YEAR}")
    print("=" * 70)
    print(f"\n{'Order Status':<15} {'Percentage':>12}")
    print("-" * 30)
    for idx, row in status_dist.iterrows():
        print(f"{row['order_status']:<15} {row['percentage']:>11.2f}%")

<a id='summary'></a>
## 6. Summary of Key Observations

### Executive Summary

This section provides a comprehensive summary of the analysis findings and strategic recommendations.

In [ ]:
# Generate executive summary
print(f"EXECUTIVE SUMMARY - {ANALYSIS_YEAR} BUSINESS PERFORMANCE")
print("=" * 70)

# Financial performance
revenue_metrics = business_report['revenue_metrics']
print(f"\nFinancial Performance:")
print(f"  Total Revenue: ${revenue_metrics['total_revenue']:,.0f}")
print(f"  Total Orders: {revenue_metrics['total_orders']:,}")
print(f"  Average Order Value: ${revenue_metrics['average_order_value']:,.0f}")

if 'revenue_growth_rate' in revenue_metrics:
    growth_direction = "increased" if revenue_metrics['revenue_growth_rate'] > 0 else "decreased"
    print(f"  Revenue Growth: {growth_direction} by {abs(revenue_metrics['revenue_growth_rate']):.1f}% vs {COMPARISON_YEAR}")

# Product performance
if 'error' not in business_report['product_performance']:
    top_category = business_report['product_performance']['top_categories'].iloc[0]
    print(f"\nProduct Performance:")
    print(f"  Top Category: {top_category['product_category_name']}")
    print(f"  Top Category Revenue: ${top_category['total_revenue']:,.0f}")
    print(f"  Market Share: {top_category['revenue_share']:.1f}%")

# Geographic performance
geo_data = business_report['geographic_performance']
if 'error' not in geo_data.columns:
    top_state = geo_data.iloc[0]
    print(f"\nGeographic Performance:")
    print(f"  Top Market: {top_state['state']}")
    print(f"  Top Market Revenue: ${top_state['revenue']:,.0f}")
    print(f"  Active Markets: {len(geo_data)} states")

# Customer experience
if 'error' not in business_report['customer_satisfaction']:
    satisfaction = business_report['customer_satisfaction']
    print(f"\nCustomer Experience:")
    print(f"  Average Rating: {satisfaction['avg_review_score']:.1f}/5.0")
    print(f"  High Satisfaction Rate: {satisfaction['score_4_plus_percentage']:.0f}% (4+ stars)")

if 'error' not in business_report['delivery_performance']:
    delivery = business_report['delivery_performance']
    print(f"  Average Delivery Time: {delivery['avg_delivery_days']:.1f} days")
    print(f"  Fast Delivery Rate: {delivery['fast_delivery_percentage']:.0f}% (3 days or less)")

In [ ]:
# Generate strategic recommendations
print("\n" + "=" * 70)
print("STRATEGIC RECOMMENDATIONS")
print("=" * 70)

recommendations = []

# Revenue-based recommendations
if 'revenue_growth_rate' in revenue_metrics:
    if revenue_metrics['revenue_growth_rate'] < 0:
        recommendations.append(
            "Address negative revenue growth through customer acquisition and retention initiatives"
        )
    elif revenue_metrics['revenue_growth_rate'] < 5:
        recommendations.append(
            "Accelerate growth through market expansion and product diversification"
        )

# Product recommendations
if 'error' not in business_report['product_performance']:
    top_5_share = business_report['product_performance']['top_categories'].head(5)['revenue_share'].sum()
    if top_5_share > 70:
        recommendations.append(
            "Diversify product portfolio to reduce dependency on top categories"
        )

# Customer experience recommendations
if 'error' not in business_report['customer_satisfaction']:
    satisfaction = business_report['customer_satisfaction']
    if satisfaction['avg_review_score'] < 4.0:
        recommendations.append(
            "Implement quality improvements to enhance customer satisfaction scores"
        )
    if satisfaction['score_1_2_percentage'] > 10:
        recommendations.append(
            "Investigate and address root causes of low satisfaction ratings"
        )

# Delivery recommendations
if 'error' not in business_report['delivery_performance']:
    delivery = business_report['delivery_performance']
    if delivery['avg_delivery_days'] > 7:
        recommendations.append(
            "Optimize logistics operations to reduce average delivery time"
        )
    if delivery['fast_delivery_percentage'] < 20:
        recommendations.append(
            "Invest in fast delivery capabilities to improve competitive position"
        )

# Geographic recommendations
if 'error' not in geo_data.columns:
    if len(geo_data) < 30:
        recommendations.append(
            "Explore expansion opportunities in underserved geographic markets"
        )

# Display recommendations
if recommendations:
    for i, rec in enumerate(recommendations, 1):
        print(f"\n{i}. {rec}")
else:
    print("\nBusiness performance is strong across all analyzed metrics")

print("\n" + "=" * 70)
print(f"Analysis completed for {ANALYSIS_YEAR}")
if COMPARISON_YEAR:
    print(f"Comparison baseline: {COMPARISON_YEAR}")
print(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

---

## Analysis Configuration Summary

This notebook provides a comprehensive, configurable framework for e-commerce business analysis.

### Key Features

- **Configurable Time Periods**: Easily adjust analysis and comparison years/months
- **Modular Architecture**: Reusable data loading and metrics calculation modules
- **Comprehensive Metrics**: Revenue, product, geographic, and customer experience analysis
- **Professional Visualizations**: Business-oriented charts with proper formatting
- **Strategic Insights**: Data-driven recommendations based on performance metrics

### How to Use

1. **Configure Analysis Parameters**: Modify ANALYSIS_YEAR, COMPARISON_YEAR, and ANALYSIS_MONTH at the top
2. **Run All Cells**: Execute the notebook to generate the complete analysis
3. **Review Outputs**: Examine metrics, visualizations, and recommendations
4. **Export Results**: Save visualizations and tables for reporting

### Extending the Analysis

- Add new metrics in the `business_metrics.py` module
- Customize visualizations in the `MetricsVisualizer` class
- Modify data loading logic in `data_loader.py` for different data sources
- Create additional analysis sections as needed

---